# [CS3244 Project on music genre classificaiton](https://github.com/markusyeo/fma)

Markus Yeo, Sunny Low

## Creation

From `raw_*.csv`, this notebook generates:
* `tracks.csv`: per-track / album / artist metadata / filtered to fma_small.
* `genres.csv`: genre hierarchy.

In [52]:
import os
import ast
import pickle

import IPython.display as ipd
import numpy as np
import pandas as pd

from dotenv import dotenv_values 
import utils
import creation

In [53]:
AUDIO_DIR = dotenv_values()['AUDIO_DIR']
BASE_DIR = os.path.abspath(os.path.dirname(AUDIO_DIR))
FMA_SMALL = os.path.join(BASE_DIR, 'fma_small')

In [54]:
AUDIO_DIR

'fma_small'

In [55]:
path = os.walk(AUDIO_DIR)
filelist = []
for root, dirs, files in path:
	for file in files:
        #append the file name to the list
		if '.mp3' in file:
			filelist.append(int(file[:-4]))
filelist = tuple(filelist)

In [56]:
filelist[:5]

(2, 5, 10, 140, 141)

In [57]:
# ./creation.py metadata
# ./creation.py data /path/to/fma/fma_full
# ./creation.py clips /path/to/fma

#!cat creation.py

In [58]:
# converters={'genres': ast.literal_eval}
tracks = pd.read_csv('./data/fma_metadata/raw_tracks.csv', index_col=0)
albums = pd.read_csv('./data/fma_metadata/raw_albums.csv', index_col=0)
artists = pd.read_csv('./data/fma_metadata/raw_artists.csv', index_col=0)
genres = pd.read_csv('./data/fma_metadata/raw_genres.csv', index_col=0)

# not_found = pickle.load(open('not_found.pickle', 'rb'))

In [59]:
for i in [98565, 98567, 98569, 99134, 108925, 133297]: 
    filelist = list(filelist)
    filelist.remove(i)
filelist = tuple(filelist)
tracks = tracks.loc[list(filelist)]
tracks = tracks.sort_index()

In [60]:
tracks.head()

,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,license_parent_id,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
track_id,,,,,,,,,,,,,,,,,,,,,
2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,5.0,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,5.0,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,NaN,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
140,61.0,The Blind Spot,http://freemusicarchive.org/music/Alec_K_Redfe...,54,Alec K. Redfearn & the Eyesores,http://freemusicarchive.org/music/Alec_K_Redfe...,http://www.aleckredfearn.com,http://i.creativecommons.org/l/by-nc-nd/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,6.0,...,NaN,0,1593,en,1299,NaN,2,NaN,Queen Of The Wires,http://freemusicarchive.org/music/Alec_K_Redfe...
141,60.0,Every Man For Himself,http://freemusicarchive.org/music/Alec_K_Redfe...,54,Alec K. Redfearn & the Eyesores,http://freemusicarchive.org/music/Alec_K_Redfe...,http://www.aleckredfearn.com,http://i.creativecommons.org/l/by-nc-nd/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,6.0,...,NaN,0,839,en,725,NaN,4,NaN,Ohio,http://freemusicarchive.org/music/Alec_K_Redfe...


In [61]:
tracks.columns

Index(['album_id', 'album_title', 'album_url', 'artist_id', 'artist_name',
       'artist_url', 'artist_website', 'license_image_file',
       'license_image_file_large', 'license_parent_id', 'license_title',
       'license_url', 'tags', 'track_bit_rate', 'track_comments',
       'track_composer', 'track_copyright_c', 'track_copyright_p',
       'track_date_created', 'track_date_recorded', 'track_disc_number',
       'track_duration', 'track_explicit', 'track_explicit_notes',
       'track_favorites', 'track_file', 'track_genres', 'track_image_file',
       'track_information', 'track_instrumental', 'track_interest',
       'track_language_code', 'track_listens', 'track_lyricist',
       'track_number', 'track_publisher', 'track_title', 'track_url'],
      dtype='object')

In [62]:
def get_fs_tids(audio_dir):
    tids = []
    for _, dirnames, files in os.walk(audio_dir):
        if dirnames == []:
            tids.extend(int(file[:-4]) for file in files)
    return tids

audio_tids = get_fs_tids(FMA_SMALL)
clips_tids = get_fs_tids(FMA_SMALL)

In [63]:
print('tracks: {} collected ({} max id)'.format(
    len(tracks), tracks.index.max()))
print('albums: {} collected ({} in tracks)'.format(
    len(albums), len(tracks['album_id'].unique())))
print('artists: {} collected ({} in tracks)'.format(
    len(artists), len(tracks['artist_id'].unique())))
print('genres: {} collected'.format(len(genres)))

print('audio: {} collected ({} not in tracks)'.format(
    len(audio_tids), len(set(audio_tids).difference(tracks.index))))
print('clips: {} collected ({} not in tracks)'.format(
    len(clips_tids), len(set(clips_tids).difference(tracks.index))))
# assert sum(tracks.index.isin(audio_tids)) == len(tracks)
# assert sum(tracks.index.isin(clips_tids)) == sum(tracks.index.isin(audio_tids))
# assert len(clips_tids) == len(tracks)

tracks: 7994 collected (155066 max id)
albums: 15234 collected (2462 in tracks)
artists: 16916 collected (2307 in tracks)
genres: 164 collected
audio: 8000 collected (6 not in tracks)
clips: 8000 collected (6 not in tracks)


In [64]:
N = 5
ipd.display(tracks.head(N))
ipd.display(albums.head(N))
ipd.display(artists.head(N))
ipd.display(genres.head(N))

,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,license_parent_id,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
track_id,,,,,,,,,,,,,,,,,,,,,
2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,5.0,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,5.0,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,NaN,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
140,61.0,The Blind Spot,http://freemusicarchive.org/music/Alec_K_Redfe...,54,Alec K. Redfearn & the Eyesores,http://freemusicarchive.org/music/Alec_K_Redfe...,http://www.aleckredfearn.com,http://i.creativecommons.org/l/by-nc-nd/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,6.0,...,NaN,0,1593,en,1299,NaN,2,NaN,Queen Of The Wires,http://freemusicarchive.org/music/Alec_K_Redfe...
141,60.0,Every Man For Himself,http://freemusicarchive.org/music/Alec_K_Redfe...,54,Alec K. Redfearn & the Eyesores,http://freemusicarchive.org/music/Alec_K_Redfe...,http://www.aleckredfearn.com,http://i.creativecommons.org/l/by-nc-nd/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,6.0,...,NaN,0,839,en,725,NaN,4,NaN,Ohio,http://freemusicarchive.org/music/Alec_K_Redfe...


,album_comments,album_date_created,album_date_released,album_engineer,album_favorites,album_handle,album_image_file,album_images,album_information,album_listens,album_producer,album_title,album_tracks,album_type,album_url,artist_name,artist_url,tags
album_id,,,,,,,,,,,,,,,,,,
1,0,11/26/2008 01:44:45 AM,1/05/2009,NaN,4,AWOL_-_A_Way_Of_Life,https://freemusicarchive.org/file/images/album...,"[{'image_id': '1955', 'image_file': 'https://f...",<p></p>,6073,NaN,AWOL - A Way Of Life,7,Album,http://freemusicarchive.org/music/AWOL/AWOL_-_...,AWOL,http://freemusicarchive.org/music/AWOL/,[]
100,0,11/26/2008 01:55:44 AM,1/09/2009,NaN,0,On_Opaque_Things,https://freemusicarchive.org/file/images/album...,"[{'image_id': '4403', 'image_file': 'https://f...",NaN,5613,NaN,On Opaque Things,4,Album,http://freemusicarchive.org/music/Bird_Names/O...,Bird Names,http://freemusicarchive.org/music/Bird_Names/,[]
1000,0,12/04/2008 09:28:49 AM,10/26/2008,NaN,0,DMBQ_Live_at_2008_Record_Fair_on_WFMU_Record_F...,https://freemusicarchive.org/file/images/album...,"[{'image_id': '31997', 'image_file': 'https://...",<p>http://blog.wfmu.org/freeform/2008/10/what-...,1092,NaN,DMBQ Live at 2008 Record Fair on WFMU Record F...,4,Live Performance,http://freemusicarchive.org/music/DMBQ/DMBQ_Li...,DMBQ,http://freemusicarchive.org/music/DMBQ/,[]
10000,0,9/05/2011 04:42:57 PM,NaN,NaN,0,Live_at_CKUT_on_Montreal_Sessions_1434,https://freemusicarchive.org/file/images/album...,"[{'image_id': '12266', 'image_file': 'https://...",<p>Live Set on the Montreal Session February 2...,1001,NaN,Live at CKUT on Montreal Sessions,1,Radio Program,http://freemusicarchive.org/music/Sundrips/Liv...,Sundrips,http://freemusicarchive.org/music/Sundrips/,[]
10001,0,9/06/2011 12:02:58 AM,1/01/2006,NaN,0,Grounds_Dream_Cosmic_Love,https://freemusicarchive.org/file/images/album...,"[{'image_id': '24091', 'image_file': 'https://...","<p>Recorded in Linnavuori, Finland, 2005 (with...",504,NaN,Ground's Dream Cosmic Love,1,Album,http://freemusicarchive.org/music/Uton/Grounds...,Uton,http://freemusicarchive.org/music/Uton/,[]


,artist_active_year_begin,artist_active_year_end,artist_associated_labels,artist_bio,artist_comments,artist_contact,artist_date_created,artist_donation_url,artist_favorites,artist_flattr_name,...,artist_location,artist_longitude,artist_members,artist_name,artist_paypal_name,artist_related_projects,artist_url,artist_website,artist_wikipedia_page,tags
artist_id,,,,,,,,,,,,,,,,,,,,,
1,2006.0,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,Brown Bum aka Choke,11/26/2008 01:42:32 AM,NaN,9,NaN,...,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,NaN,The list of past projects is 2 long but every1...,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,NaN,['awol']
10,NaN,NaN,"Mistletone, Marriage Records","<p>""Lucky Dragons"" means any recorded or perfo...",3,Lukey Dargons,11/26/2008 01:43:35 AM,http://glaciersofnice.com/shop/,111,NaN,...,"Los Angeles, CA",-118.243685,Luke Fischbeck\nSarah Rara,Lucky Dragons,NaN,NaN,http://freemusicarchive.org/music/Lucky_Dragons/,http://hawksandsparrows.org/,NaN,['lucky dragons']
100,2004.0,NaN,"Captcha Records (HBSP-2X), Pickled Egg (Europe)","<p><span style=""font-family:Verdana, Geneva, A...",1,Chris Kalis,11/26/2008 02:05:22 AM,NaN,8,NaN,...,"Chicago, IL",-87.629798,"Chris Kalis, Harry Brenner, Scott McGaughey, B...",Chandeliers,NaN,"Killer Whales, \nMichael Columbia\nMandate\nMr...",http://freemusicarchive.org/music/Chandeliers/,thechandeliers.com,NaN,['chandeliers']
1000,NaN,NaN,NaN,"<p><a href=""http://marzipanmarzipan.com"">Marzi...",0,NaN,12/04/2008 09:24:35 AM,NaN,0,NaN,...,NaN,12.567380,NaN,Marzipan Marzipan,NaN,NaN,http://freemusicarchive.org/music/Marzipan_Mar...,https://soundcloud.com/marzipanmarzipan,NaN,[]
10000,NaN,NaN,NaN,"<p><span style=""font-family:'Times New Roman',...",0,NaN,1/21/2011 02:11:31 PM,NaN,1,NaN,...,NaN,NaN,Jack Hertz\nPHOBoS\nBlue Hell,"Jack Hertz, PHOBoS, Blue Hell",NaN,NaN,http://freemusicarchive.org/music/Jack_Hertz_P...,http://surrism.phonoethics.com/surrism-phonoet...,NaN,['jack hertz phobos blue hell']


,genre_color,genre_handle,genre_parent_id,genre_title
genre_id,,,,
1,#006666,Avant-Garde,38.0,Avant-Garde
2,#CC3300,International,NaN,International
3,#000099,Blues,NaN,Blues
4,#990099,Jazz,NaN,Jazz
5,#8A8A65,Classical,NaN,Classical


## 2 Format metadata

Todo:
* Sanitize values, e.g. list of words for tags, valid links in `artist_wikipedia_page`, remove html markup in free-form text.
    * Clean tags. E.g. some tags are just artist names.
* Fill metadata about encoding: length, number of samples, sample rate, bit rate, channels (mono/stereo), 16bits?.
* Update duration from audio
    * 2624 is marked as 05:05:50 (18350s) although it is reported as 00:21:15.15 by ffmpeg.
    * 112067: 3714s --> 01:59:55.06, 112808: 3718s --> 01:59:59.56
    * ffmpeg: Estimating duration from bitrate, this may be inaccurate
    * Solution, decode the complete mp3: `ffmpeg -i input.mp3 -f null -`

In [65]:
df, column = tracks, 'tags'
null = sum(df[column].isnull())
print('{} null, {} non-null'.format(null, df.shape[0] - null))
df[column].value_counts().head(10)

0 null, 7994 non-null


[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       6636
['holiday music']                                                                                                                                                                                                                                                                                                                         

### 2.1 Tracks

In [66]:
#Helper functions
def convert_datetime(df, column, format=None):
    df[column] = pd.to_datetime(df[column], infer_datetime_format=True, format=format)

def convert_genres(genres):
    genres = ast.literal_eval(genres)
    return [int(genre['genre_id']) for genre in genres]

In [67]:
drops = [
    'artist_website', 'license_image_file', 'license_image_file_large', 
    'license_parent_id', 'license_title', 'license_url', 'track_copyright_p', 'track_copyright_c', 
    'track_file', 'track_image_file',  # used to download only
    'track_url', 'album_url', 'artist_url',  # only relevant on website
    'track_composer', 'track_lyricist', 'track_publisher',  # present for ~4000, <1000 and <2000 tracks
    'track_disc_number',  # different from 1 for <1000 tracks
    'track_explicit', 'track_explicit_notes',  # present for <4000 tracks
    'track_instrumental'  # ~6000 tracks have a 1, there is an instrumental genre
]
tracks.drop(drops, axis=1, inplace=True)
tracks.rename(columns={'tags': 'track_tags'}, inplace=True)

In [68]:
tracks['track_duration'] = tracks['track_duration'].map(creation.convert_duration)
convert_datetime(tracks, 'track_date_created')
convert_datetime(tracks, 'track_date_recorded')
tracks['album_id'].fillna(-1, inplace=True)
tracks['track_bit_rate'].fillna(-1, inplace=True)
tracks = tracks.astype({'album_id': int, 'track_bit_rate': int})
tracks['track_genres'].fillna('[]', inplace=True)
tracks['track_genres'] = tracks['track_genres'].map(convert_genres)

In [69]:
tracks.columns

Index(['album_id', 'album_title', 'artist_id', 'artist_name', 'track_tags',
       'track_bit_rate', 'track_comments', 'track_date_created',
       'track_date_recorded', 'track_duration', 'track_favorites',
       'track_genres', 'track_information', 'track_interest',
       'track_language_code', 'track_listens', 'track_number', 'track_title'],
      dtype='object')

### 2.2 Albums

In [70]:
albums.columns

Index(['album_comments', 'album_date_created', 'album_date_released',
       'album_engineer', 'album_favorites', 'album_handle', 'album_image_file',
       'album_images', 'album_information', 'album_listens', 'album_producer',
       'album_title', 'album_tracks', 'album_type', 'album_url', 'artist_name',
       'artist_url', 'tags'],
      dtype='object')

In [71]:
drop = [
    'artist_name', 'album_url', 'artist_url',  # in tracks already (though it can be different)
    'album_handle',
    'album_image_file', 'album_images',  # todo: shall be downloaded
    #'album_producer', 'album_engineer',  # present for ~2400 albums only
]
albums.drop(drop, axis=1, inplace=True)
albums.rename(columns={'tags': 'album_tags'}, inplace=True)
convert_datetime(albums, 'album_date_created')
convert_datetime(albums, 'album_date_released')

In [72]:
albums.columns

Index(['album_comments', 'album_date_created', 'album_date_released',
       'album_engineer', 'album_favorites', 'album_information',
       'album_listens', 'album_producer', 'album_title', 'album_tracks',
       'album_type', 'album_tags'],
      dtype='object')

### 2.3 Artists

In [73]:
artists.columns

Index(['artist_active_year_begin', 'artist_active_year_end',
       'artist_associated_labels', 'artist_bio', 'artist_comments',
       'artist_contact', 'artist_date_created', 'artist_donation_url',
       'artist_favorites', 'artist_flattr_name', 'artist_handle',
       'artist_image_file', 'artist_images', 'artist_latitude',
       'artist_location', 'artist_longitude', 'artist_members', 'artist_name',
       'artist_paypal_name', 'artist_related_projects', 'artist_url',
       'artist_website', 'artist_wikipedia_page', 'tags'],
      dtype='object')

In [74]:
drop = [
    'artist_website', 'artist_url',  # in tracks already (though it can be different)
    'artist_handle',
    'artist_image_file', 'artist_images',  # todo: shall be downloaded
    'artist_donation_url', 'artist_paypal_name', 'artist_flattr_name',  # ~1600 & ~400 & ~70, not relevant
    'artist_contact',  # ~1500, not very useful data
    'artist_active_year_begin', 'artist_active_year_end',  # ~1400, ~500 only
    'artist_associated_labels',  # ~1000
    'artist_related_projects',  # only ~800, but can be combined with bio
]
artists.drop(drop, axis=1, inplace=True)
artists.rename(columns={'tags': 'artist_tags'}, inplace=True)

In [75]:
# convert_datetime(artists, 'artist_date_created')
# for column in ['artist_active_year_begin', 'artist_active_year_end']:
#     artists[column].replace(0.0, np.nan, inplace=True)
#     convert_datetime(artists, column, format='%Y.0')

In [76]:
artists.columns

Index(['artist_bio', 'artist_comments', 'artist_date_created',
       'artist_favorites', 'artist_latitude', 'artist_location',
       'artist_longitude', 'artist_members', 'artist_name',
       'artist_wikipedia_page', 'artist_tags'],
      dtype='object')

### 2.4 Merge DataFrames

In [77]:
tracks = tracks.merge(albums, left_on='album_id', right_index=True, sort=False, how='left', suffixes=('', '_dup'))

n = tracks['album_title_dup'].isnull().sum()
print('{} tracks without extended album information ({} tracks without album_id)'.format(n, sum(tracks['album_id'] == -1)))
# assert sum(tracks['album_id'].isin(not_found['albums'])) == n
# assert sum(tracks['album_title'] != tracks['album_title_dup']) == n

tracks.drop('album_title_dup', axis=1, inplace=True)
# assert not any('dup' in col for col in tracks.columns)

0 tracks without extended album information (0 tracks without album_id)


In [78]:
# Album artist can be different than track artist. Keep track artist.
#tracks[tracks['artist_name'] != tracks['artist_name_dup']].select(lambda x: 'artist_name' in x, axis=1)

In [79]:
tracks = tracks.merge(artists, left_on='artist_id', right_index=True, sort=False, how='left', suffixes=('', '_dup'))

n = tracks['artist_name_dup'].isnull().sum()
print('{} tracks without extended artist information'.format(n))
# assert sum(tracks['artist_id'].isin(not_found['artists'])) == n
assert sum(tracks['artist_name'] != tracks[('artist_name_dup')]) == n

tracks.drop('artist_name_dup', axis=1, inplace=True)
# assert not any('dup' in col for col in tracks.columns)

0 tracks without extended artist information


In [80]:
columns = []
for name in tracks.columns:
    names = name.split('_')
    columns.append((names[0], '_'.join(names[1:])))
tracks.columns = pd.MultiIndex.from_tuples(columns)
# assert all(label in ['track', 'album', 'artist'] for label in tracks.columns.get_level_values(0))

In [81]:
# Todo: fill other columns ?
tracks['album', 'tags'].fillna('[]', inplace=True)
tracks['artist', 'tags'].fillna('[]', inplace=True)

columns = [('album', 'favorites'), ('album', 'comments'), ('album', 'listens'), ('album', 'tracks'),
           ('artist', 'favorites'), ('artist', 'comments')]
for column in columns:
    tracks[column].fillna(-1, inplace=True)
columns = {column: int for column in columns}
tracks = tracks.astype(columns)

## 3 Data cleaning

Todo: duplicates (metadata and audio)

In [82]:
def keep(index, df):
    old = len(df)
    df = df.loc[index]
    new = len(df)
    print('{} lost, {} left'.format(old - new, new))
    return df

tracks = keep(tracks.index, tracks)

0 lost, 7994 left


Errors from the `features.py` script.
* IndexError('index 0 is out of bounds for axis 0 with size 0',)
    * ffmpeg: Header missing
    * ffmpeg: Could not find codec parameters for stream 0 (Audio: mp3, 0 channels, s16p): unspecified frame size. Consider increasing the value for the 'analyzeduration' and 'probesize' options
    * tids: 117759
* NoBackendError()
    * ffmpeg: Format mp3 detected only with low score of 1, misdetection possible!
    * tids: 80015, 115235
* UserWarning('Trying to estimate tuning from empty frequency set.',)
    * librosa error
    * tids: 1440, 26436, 38903, 57603, 62095, 62954, 62956, 62957, 62959, 62971, 86079, 96426, 104623, 106719, 109714, 114501, 114528, 118003, 118004, 127827, 130298, 130296, 131076, 135804, 154923
* ParameterError('Filter pass-band lies beyond Nyquist',)
    * librosa error
    * tids: 152204, 28106, 29166, 29167, 29169, 29168, 29170, 29171, 29172, 29173, 29179, 43903, 56757, 59361, 75461, 92346, 92345, 92347, 92349, 92350, 92351, 92353, 92348, 92352, 92354, 92355, 92356, 92358, 92359, 92361, 92360, 114448, 136486, 144769, 144770, 144771, 144773, 144774, 144775, 144778, 144776, 144777

In [83]:
# Too short a clip, known issue in wiki page
FAILED = [98565, 98567, 98569, 99134, 108925, 133297]
tracks = keep(tracks.index.difference(FAILED), tracks)

0 lost, 7994 left


In [84]:
sum(tracks['track', 'title'].duplicated())

232

## 4 Genres

In [85]:
genres.drop(['genre_handle', 'genre_color'], axis=1, inplace=True)
genres.rename(columns={'genre_parent_id': 'parent', 'genre_title': 'title'}, inplace=True)

In [86]:
genres['parent'].fillna(0, inplace=True)
genres = genres.astype({'parent': int})

In [87]:
# 13 (Easy Listening) has parent 126 which is missing
# --> a root genre on the website, although not in the genre menu
genres.at[13, 'parent'] = 0

# 580 (Abstract Hip-Hop) has parent 1172 which is missing
# --> listed as child of Hip-Hop on the website
genres.at[580, 'parent'] = 21

# 810 (Nu-Jazz) has parent 51 which is missing
# --> listed as child of Easy Listening on website
genres.at[810, 'parent'] = 13

# 763 (Holiday) has parent 763 which is itself
# --> listed as child of Sound Effects on website
genres.at[763, 'parent'] = 16

# Todo: should novelty be under Experimental? It is alone on website.

In [88]:
# Genre 806 (hiphop) should not exist. Replace it by 21 (Hip-Hop).
print('{} tracks have genre 806'.format(
    sum(tracks['track', 'genres'].map(lambda genres: 806 in genres))))
def change_genre(genres):
    return [genre if genre != 806 else 21 for genre in genres]
tracks['track', 'genres'] = tracks['track', 'genres'].map(change_genre)
genres.drop(806, inplace=True)

5 tracks have genre 806


In [89]:
def get_parent(genre, track_all_genres=None):
    parent = genres.at[genre, 'parent']
    if track_all_genres is not None:
        track_all_genres.append(genre)
    return genre if parent == 0 else get_parent(parent, track_all_genres)

# Get all genres, i.e. all genres encountered when walking from leafs to roots.
def get_all_genres(track_genres):
    track_all_genres = list()
    for genre in track_genres:
        get_parent(genre, track_all_genres)
    return list(set(track_all_genres))

tracks['track', 'genres_all'] = tracks['track', 'genres'].map(get_all_genres)

In [90]:
# Number of tracks per genre.
def count_genres(subset=tracks.index):
    count = pd.Series(0, index=genres.index)
    for _, track_all_genres in tracks.loc[subset, ('track', 'genres_all')].items():
        for genre in track_all_genres:
            count[genre] += 1
    return count

genres['#tracks'] = count_genres()
genres[genres['#tracks'] == 0]

,parent,title,#tracks
genre_id,,,
3,0,Blues,0
4,0,Jazz,0
5,0,Classical,0
7,20,Comedy,0
8,0,Old-Time / Historic,0
9,0,Country,0
11,14,Disco,0
13,0,Easy Listening,0
14,0,Soul-RnB,0


In [91]:
def get_top_genre(track_genres):
    top_genres = set(genres.at[genres.at[genre, 'top_level'], 'title'] for genre in track_genres)
    return top_genres.pop() if len(top_genres) == 1 else np.nan

# Top-level genre.
genres['top_level'] = genres.index.map(get_parent)
tracks['track', 'genre_top'] = tracks['track', 'genres'].map(get_top_genre)

In [92]:
genres.head(10)

,parent,title,#tracks,top_level
genre_id,,,,
1,38,Avant-Garde,211,38
2,0,International,1000,2
3,0,Blues,0,3
4,0,Jazz,0,4
5,0,Classical,0,5
6,38,Novelty,90,38
7,20,Comedy,0,20
8,0,Old-Time / Historic,0,8
9,0,Country,0,9


## 5 Small Dataset

Main characteristic: genre balanced.

Choices:
* 8 genres with 1000 tracks --> 8,000 tracks

In [93]:
fma_small = pd.DataFrame(tracks)

In [94]:
SUBSETS = ['small']
tracks['set', 'subset'] = pd.Series().astype(pd.CategoricalDtype( categories=SUBSETS, ordered=True))
tracks.loc[fma_small.index, ('set', 'subset')] = 'small'

C:\Users\Markus Yeo\AppData\Local\Temp\ipykernel_1540\178925839.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tracks['set', 'subset'] = pd.Series().astype(pd.CategoricalDtype( categories=SUBSETS, ordered=True))


## 6 Splits: training, validation, test

Take into account:
* Artists may only appear on one side.
* Stratification: ideally, all characteristics (#tracks per artist, duration, sampling rate, information, bio) and targets (genres, tags) should be equally distributed.

In [106]:
tracks.loc[training.index, ('set', 'split')]

track_id
127661    training
114239    training
119895    training
125815    training
128471    training
            ...   
125825    training
114238    training
125809    training
129184    training
125821    training
Name: (set, split), Length: 208, dtype: category
Categories (3, object): ['training', 'test', 'validation']

In [150]:
from fast_ml.model_development import train_valid_test_split
training, _, validation, _, test, _ = train_valid_test_split(temp_tracks, train_size=0.8, valid_size=0.1, test_size=0.1, target=('track','genre_top'))

In [179]:
splits

[278, 21, 34]

In [185]:
np.split(temp_tracks.sample(frac=1, random_state=42), [splits[0], splits[1]+splits[0]])


[          album                                       artist  \
              id                                 title     id   
 track_id                                                       
 50955      9616                  Contemporary Sampler  11431   
 141144    21144                             MUL / DIV  20985   
 92953     15344                      Tamara and Demon  17278   
 123487    18676                           Chapter One  20655   
 86038     14503                 At the House of Dolls  16532   
 ...         ...                                   ...    ...   
 141137    21223    The Giant Illusion (A Compilation)  22533   
 134580    20201                Northern City's Secret  20760   
 123763    18727                     Thoughts On Weeks  20678   
 118952    18211                                    Be  20112   
 63291     11468  I Was A Teenage Strangler Soundtrack  13346   
 
                                                 \
                                     

In [188]:
def get_genre_id(genre):
    return genres[genres['title']==genre].index[0]

for genre in [get_genre_id(g) for g in tracks[('track', 'genre_top')]]:
    tracks['genre', genres.at[genre, 'title']] = tracks['track', 'genres_all'].map(lambda genres: genre in genres)
    
SPLITS = ('training', 'test', 'validation')
PERCENTAGES = (0.8, 0.1, 0.1)
tracks['set', 'split'] = pd.Series(dtype = pd.CategoricalDtype(categories=SPLITS))

genre_list = list(tracks[('track', 'genre_top')].unique())

while True:
    if len(genre_list) == 0:
        break

    # Choose most constrained genre, i.e. genre with the least unassigned artists.
    tracks_unsplit = tracks['set', 'split'].isnull()
    temp = tracks.loc[tracks.index.intersection(tracks_unsplit.index, sort=False)].set_index(('artist', 'id'), append=True)['genre']
    count = temp.groupby(level=1).sum().astype(bool).sum()
    genre = count[genre_list].idxmin()
    if genre in genre_list:
        genre_list.remove(genre)
    
    # Given genre, select artists.
    tracks_genre = tracks[tracks[('genre', genre)] == True]
    
    artists = tracks.loc[temp.index.get_level_values(level='track_id').intersection(tracks_genre.index, sort=False), ('artist', 'id')].value_counts()
    #print('-->', genre, len(artists))

    current = {split: np.count_nonzero(tracks.index.intersection(tracks_genre.index, sort=False).intersection(tracks[tracks['set', 'split'] == split].index)) for split in SPLITS}
    # print(current)

    # Only classify artists with more than 10 songs into training, validation, and test sets.
    remainder = []
    for artist, count in artists.items():
        temp_tracks = tracks.loc[tracks.loc[tracks['artist', 'id'] == artist].index.intersection(tracks_genre.index, sort=False)]
        if len(temp_tracks) >= 10:
            training, validation, test = np.split(temp_tracks.sample(frac=1, random_state=42), [int(.8*len(temp_tracks)), int(.9*len(temp_tracks))])
            current['training'] += len(training)
            tracks.loc[training.index, ('set', 'split')] = 'training'
            current['test'] += len(test)
            tracks.loc[test.index, ('set', 'split')] = 'test'
            current['validation'] += len(validation)
            tracks.loc[validation.index, ('set', 'split')] = 'validation'
        else:
            continue
    temp_tracks = tracks_genre[tracks[('set','split')].isnull()]
    splits = [len(tracks_genre)*percentage for percentage in PERCENTAGES]
    splits = [int(split) - current[SPLITS[i]] for i, split in enumerate(splits)]
    print(splits)
    training, validation, test = np.split(temp_tracks.sample(frac=1, random_state=42), [splits[0], splits[2]+splits[0]])
    current['training'] += len(training)
    tracks.loc[training.index, ('set', 'split')] = 'training'
    current['test'] += len(test)
    tracks.loc[test.index, ('set', 'split')] = 'test'
    current['validation'] += len(validation)
    tracks.loc[validation.index, ('set', 'split')] = 'validation'
    
    print(current)
# Not needed any more.
tracks.drop('genre', axis=1, level=0, inplace=True)

[278, 21, 34]
{'training': 800, 'test': 100, 'validation': 100}
[422, 39, 52]
{'training': 797, 'test': 101, 'validation': 99}
[504, 55, 65]
{'training': 800, 'test': 100, 'validation': 100}
[490, 47, 61]
{'training': 800, 'test': 100, 'validation': 100}
[531, 56, 67]
{'training': 800, 'test': 100, 'validation': 100}
[589, 63, 75]
{'training': 799, 'test': 101, 'validation': 99}
[687, 77, 86]
{'training': 799, 'test': 101, 'validation': 99}
[659, 74, 82]
{'training': 799, 'test': 101, 'validation': 99}


In [189]:
tracks['set','split'].value_counts()

training      6394
test           804
validation     796
Name: (set, split), dtype: int64

## 7 Store

In [191]:
os.chdir('./data/fma_metadata')

In [192]:
for dataset in 'tracks', 'genres':
    eval(dataset).sort_index(axis=0, inplace=True)
    eval(dataset).sort_index(axis=1, inplace=True)
    params = dict(float_format='%.10f') if dataset == 'echonest' else dict()
    eval(dataset).to_csv(dataset + '.csv', **params)

In [ ]:
tracks.columns

In [ ]:
# ./creation.py normalize /path/to/fma
# ./creation.py zips /path/to/fma

## 8 Description

In [ ]:
import utils
tracks = utils.load('tracks.csv')
tracks.dtypes

In [ ]:
N = 5
ipd.display(tracks['track'].head(N))
ipd.display(tracks['album'].head(N))
ipd.display(tracks['artist'].head(N))